## <font style="font-family:roboto;color:#455e6c"> Introduction to Simulations with Pyiron </font>  

<div class="admonition note" name="html-admonition" style="background:#e3f2fd; padding: 10px">
<font style="font-family:roboto;color:#455e6c"> <b> StahlDigital Tutorial: Creating and Running Simulations for Steel Development </b> </font> </br>
<font style="font-family:roboto;color:#455e6c"> 25 April 2024 </font>
</div>

Before the excercise, you should:

* Be familiar with python especially with numerical libraries like numpy and plotting tools like matplotlib
* Understand how Jupyter Notebook works

The aim of this exercise is to make you familiar with:

* A general overview of pyiron functionality
* How to set up atomic structures and run simulation codes through pyiron

### <font style="font-family:roboto;color:#455e6c"> Import necessary libraries </font>  
As a first step we import the libraries [numpy](http://www.numpy.org/) for data analysis and [matplotlib](https://matplotlib.org/) for visualization.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pylab as plt

Fundamentally, we only need to import one module from `pyiron`: the `Project` class

In [ ]:
from pyiron import Project

The Project object introduced below is central in pyiron. It allows to name the project as well as to derive all other objects such as structures, jobs etc. without having to import them. Thus, by code completion *Tab* the respective commands can be found easily.

We now create a pyiron Project named *'intro_pyiron'.*

### <font style="font-family:roboto;color:#455e6c"> Working with atomistic structures </font>  

#### <font style="font-family:roboto;color:#455e6c"> Creation of a project instance </font>  

In [ ]:
pr = Project("intro_pyiron")

The project name also applies for the directory that is created for the project. All data generated by this `Project` object resides in this directory.

In [ ]:
pr.path

In [ ]:
pr

The `groups` and `nodes` will be populated later, as we add jobs and sub project to it.

#### <font style="font-family:roboto;color:#455e6c"> Creating atomic structures </font>  

Every atomistic simulation needs an atomic structure. For more details on generating and manipulating structures, please have a look at our [structures example](https://pyiron.readthedocs.io/en/latest/source/notebooks/structures.html). In this section however, we show how to generate and manipulate bulk crystals. pyiron's structure class is derived from the popular [ASE](https://wiki.fysik.dtu.dk/ase/ase/build/build.html) package and any `ASE` function to manipulate structures can also be applied here.

Creating a bulk bcc cubic unitcell

In [ ]:
unitcell_bcc_Fe = pr.create.structure.bulk('Fe', cubic=True, crystalstructure='bcc', a=2.84)
unitcell_bcc_Fe

In [ ]:
unitcell_bcc_Fe.plot3d()

Creating a super cell.

In [ ]:
supercell_bcc_Fe_4_4_4 = unitcell_bcc_Fe.repeat([4, 4, 4])

In [ ]:
supercell_bcc_Fe_4_4_4.plot3d()

In [ ]:
supercell_bcc_Fe_4_4_4.get_chemical_formula()

Replace one atom in the supercell with Ni

In [ ]:
supercell_bcc_Fe127Ni1 = supercell_bcc_Fe_4_4_4.copy()
supercell_bcc_Fe127Ni1[0] = "Ni" # Replacing the first Fe atom with Ni

In [ ]:
supercell_bcc_Fe127Ni1.get_chemical_formula()

In [ ]:
supercell_bcc_Fe127Ni1.plot3d()

### <font style="font-family:roboto;color:#455e6c"> Running an atomistic calculation using interatomic potentials (with LAMMPS) </font>  


Once we have an atomic structure, we can set up a simulation "job" of any atomistic simulation that is intergrated within pyiron. In this section, we are going to use the popular code [LAMMPS](https://lammps.sandia.gov/). For more details on how to steup a pyiron `job` have a look at [here](https://pyiron.readthedocs.io/en/latest/source/notebooks/first_steps.html)

In [ ]:
# Create a job
job_lammps = pr.create.job.Lammps("lammps_simulation")

Every atomistic simulation code needs an input atomic structure. We use the Fe supercell structure we created earlier

In [ ]:
# Assign an atomic structure to the job
job_lammps.structure = pr.create.structure.bulk('Fe', cubic=True, a=2.84).repeat(4)

Once the structure is assigned, an appropriate potential should also be chosen. This list of available for the structure containing Fe can be found below.  This list originates from the [NIST Interatomic Potential Database](https://www.ctcms.nist.gov/potentials/).

In [ ]:
# See available potentials
job_lammps.list_potentials()[:12]

In [ ]:
# Choose one of these potentials
job_lammps.potential = '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1'

At this stage, the computational parameters for the simulation needs to be specified. pyiron parses generic computational parameters into code specific parameters allowing for an easy transition between simulation codes

In [ ]:
# specify calculation details: in this case, a molecular dynamics simulation
job_lammps.calc_md(temperature=600, n_print=50,  n_ionic_steps=10000)

We can now see how pyiron sets-up the corresponding LAMMPS input

In [ ]:
job_lammps.input.control

Once the `run()` commmand is called, pyiron creates necessary input files, calls the simulation code, and finally parses and stores the output.

In [ ]:
job_lammps.run()

When printing the project, the saved job will also appear under `nodes` now.

In [ ]:
pr

You can get a quick overview with the `job_table` method.

In [ ]:
pr.job_table()

Once it is finished we can access the parsed output.

In [ ]:
job_lammps['output']

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(job_lammps['output/generic/temperature'], "-")
plt.xlabel("MD steps")
plt.ylabel("Temperature (K)")
plt.show();

In [ ]:
job_lammps.animate_structures()

### <font style="font-family:roboto;color:#455e6c"> Running an atomistic simulation to calculate elastic tensors of bcc iron </font>  

We will use pyiron `Lammps` and `ElasticTensor` jobs to calculate the elastic tensors.

In [ ]:
# First, we create and optimizate the structure as we did before
job_mini = pr.create.job.Lammps(job_name='lammps_mini_job')
job_mini.structure = pr.create.structure.bulk("Fe", "bcc", cubic=True)
job_mini.potential = '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1'
job_mini.calc_minimize(pressure=0.0, n_print=1)
job_mini.run()

#### <font style="font-family:roboto;color:#455e6c"> Create and run a job to calculate elastic tensor </font> 


Now, we need a reference job for elastic tensor calculation. We will create a static lammps job as reference job. 

In [ ]:
job_ref = pr.create.job.Lammps(job_name='lammps_ref_job')
job_ref.structure = job_mini.get_structure(-1) # Assign the already optimized structure to this job
job_ref.potential = '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1'
job_ref.calc_static()

Next, we will create a `ElasticTensor` job and assign `job_ref` to this job

In [ ]:
job_elastic = pr.create.job.ElasticTensor(job_name="elastic_tensor_job")
job_elastic.ref_job = job_ref

In [ ]:
job_elastic.input

In [ ]:
# Call the run function
job_elastic.run()

Collect the calculated elastic_tensor

In [ ]:
elastic_tensor = job_elastic['output/elastic_tensor']

In [ ]:
print(elastic_tensor)

The elastic tensor is a 6x6 matrix as show below:
\begin{equation}
\begin{pmatrix}
  C_{11}       & C_{12}   & C_{12}   & 0  & 0 & 0  \\
  C_{12}       & C_{11}   & C_{12}   & 0  & 0 & 0  \\
  C_{12}       & C_{12}   & C_{11}   & 0  & 0 & 0  \\
  0            & 0        & 0        & C_{44}  & 0 & 0  \\
  0            & 0        & 0        & 0       & C_{44} & 0  \\
  0            & 0        & 0        & 0       & 0 & C_{44}  \\
\end{pmatrix}
\end{equation}

In [ ]:
# Display the 6x6 elasticity tensor
plt.imshow(elastic_tensor)
plt.colorbar().set_label('Elastic constants GPa')
plt.show();

### <font style="font-family:roboto;color:#455e6c"> Software used in this notebook </font>  

- [pyiron_atomistics](https://github.com/pyiron/pyiron_atomistics)
- [LAMMPS](https://www.lammps.org/)
- [DAMASK](https://damask.mpie.de/release)